# Steam Palette Extractor
Reference: https://github.com/woctezuma/steam-palette-extractor

## Install Python packages

In [ ]:
%cd /content
!git clone https://github.com/woctezuma/steam-palette-extractor.git
%cd /content/steam-palette-extractor
!git pull
%pip install -qq -r requirements.txt

## Download images from Steam (only the first time)

In [ ]:
from src.constants import APPID_FNAME

GITHUB_URL = "https://github.com/woctezuma/steam-palette-extractor/releases"

!curl -OL {GITHUB_URL}/download/games/{APPID_FNAME}

In [ ]:
from src.constants import IMG_FOLDER
from src.download_utils import write_to_text_file
from src.utils import get_app_ids

TEMPORARY_FILE = "myimglist.txt"

write_to_text_file(get_app_ids(), fname=TEMPORARY_FILE)

# The download process took ~ 30 minutes.
# Out of 95,800 images, 92,249 were successfully downloaded.
# The output folder uses ~ 8 GB of disk space.
!echo img2dataset --url_list={TEMPORARY_FILE} --output_folder={IMG_FOLDER} --resize_mode=no

## Check the content of the image folder

In [ ]:
from src.filter_utils import prepare_filtered_files
from src.utils import get_app_ids, get_test_fnames

test_fnames = get_test_fnames(f'{IMG_FOLDER}/')

prepare_filtered_files(get_app_ids(), test_fnames)

## Compute the palette for each Steam game

In [ ]:
from src.extract_utils import extract_from_scratch

pre_computed_palettes = extract_from_scratch(test_fnames)

## Load pre-computed data

In [ ]:
from src.constants import FILTERED_APP_IDS_FNAME, PALETTE_FNAME, HSV_PALETTE_FNAME, CIELAB_PALETTE_FNAME, CIELUV_PALETTE_FNAME

!curl -OL {GITHUB_URL}/download/colors/{FILTERED_APP_IDS_FNAME}
!curl -OL {GITHUB_URL}/download/colors/{PALETTE_FNAME}
!curl -OL {GITHUB_URL}/download/colors/{HSV_PALETTE_FNAME}
!curl -OL {GITHUB_URL}/download/colors/{CIELAB_PALETTE_FNAME}
!curl -OL {GITHUB_URL}/download/colors/{CIELUV_PALETTE_FNAME}

In [ ]:
from src.utils import get_filtered_app_ids, get_pre_computed_palettes

pre_computed_app_ids = get_filtered_app_ids()
palettes_rgb = get_pre_computed_palettes()

In [ ]:
import torch

from src.color_utils import to_color_space_sequential

try:
  palettes_hsv = get_pre_computed_palettes(HSV_PALETTE_FNAME)
except FileNotFoundError:
  # The conversion process takes ~ 15 minutes, as it is not vectorized.
  palettes_hsv = to_color_space_sequential(palettes_rgb, "hsv")
  torch.save(palettes_hsv, HSV_PALETTE_FNAME)

In [ ]:
try:
  palettes_lab = get_pre_computed_palettes(CIELAB_PALETTE_FNAME)
except FileNotFoundError:
  # The conversion process takes several minutes, as it is not vectorized.
  palettes_lab = to_color_space_sequential(palettes_rgb, "cielab")
  torch.save(palettes_lab, CIELAB_PALETTE_FNAME)

In [ ]:
try:
  palettes_luv = get_pre_computed_palettes(CIELAB_PALETTE_FNAME)
except FileNotFoundError:
  # The conversion process takes several minutes, as it is not vectorized.
  palettes_luv = to_color_space_sequential(palettes_rgb, "cieluv")
  torch.save(palettes_luv, CIELUV_PALETTE_FNAME)

## Load data intended to evaluate the results

In [ ]:
from src.constants import APPID_FNAME, POPULAR_APPIDS_FNAME, SOLUTIONS_FNAME

GITHUB_URL_FOR_POPULARITY = "https://github.com/woctezuma/steam-popular-appids/releases"

!curl -OL {GITHUB_URL}/download/solutions/{SOLUTIONS_FNAME}
!curl -o {POPULAR_APPIDS_FNAME} -L {GITHUB_URL_FOR_POPULARITY}/download/data/{APPID_FNAME}

In [ ]:
from src.utils import get_egs_solutions, get_popular_appids

egs_solutions = get_egs_solutions()
popular_appids = get_popular_appids()

## Run the workflow

In [ ]:
from src.constants import get_default_params, COLOR_SPACES, PALETTE_DISTANCES

print(f"Possible color spaces: {COLOR_SPACES}")
print(f"Possible palette distances: {PALETTE_DISTANCES}")

params = get_default_params()

params["color_space"] = "linear_hsv"
params["palette_distance"] = "custom_hausdorff_distance"

params["exponent"] = 1.0
params["factor"] = 0.05

params["topk"] = 1e5
params["max_num_popular_app_ids"] = 1e5

print(params)

### Define the target

In [ ]:
from src.image_utils import prepare_image
from src.url_utils import from_gift_to_egs_url

gift_index = 12
gift = egs_solutions["gift"][gift_index]

path_or_url = from_gift_to_egs_url(egs_solutions, gift)
reference_colors = prepare_image(path_or_url, params)

### Check the ground truth

In [ ]:
from src.download_utils import get_image_url
from src.image_utils import prepare_image
from src.score_utils import compute_distance_between_palettes

# There can be several appIDs for different editions of a game, e.g. GOTY.
for ground_truth_app_id in gift["appids"]:
  path_or_url = get_image_url(ground_truth_app_id)
  ground_truth_colors = prepare_image(path_or_url, params)

  distance = compute_distance_between_palettes(
      reference_colors,
      ground_truth_colors,
      params,
      ).item()

  print(f'\tappID: {ground_truth_app_id} ; distance: {distance:.2f}')

### Check all

#### Select the palettes

In [ ]:
from src.color_utils import change_hsv_coordinates_vectorized

if params["color_space"].endswith("hsv"):
  print("HSV")
  pre_computed_palettes = palettes_hsv
elif params["color_space"].endswith("lab"):
  print("CIE LAB")
  pre_computed_palettes = palettes_lab
elif params["color_space"].endswith("luv"):
  print("CIE LUV")
  pre_computed_palettes = palettes_luv
else:
  print("RGB")
  pre_computed_palettes = palettes_rgb

if params["color_space"].startswith("linear"):
  print("Linearization")
  pre_computed_palettes = change_hsv_coordinates_vectorized(pre_computed_palettes)

#### Constrain the results to popular apps

In [ ]:
from src.optimize_utils import get_subset_of_pre_computed_data
from src.trim_utils import trim_popular_appids

if params["max_num_popular_app_ids"]:
  test_app_ids = trim_popular_appids(popular_appids, params)
else:
  test_app_ids = pre_computed_app_ids

palettes_subset, app_ids_subset = get_subset_of_pre_computed_data(
    pre_computed_palettes,
    pre_computed_app_ids,
    test_app_ids,
)

# We constrain the number of appIDs (originally ~ 100k) to focus on games which
# may be able to attract the attention of Epic Games in order to strike a deal
# for a giveaway.
# This step is not mandatory, but it should help to make the whole process
# faster, and make the game of interest appear at lower ranks in the results.
# This means that it is easier to manually parse the results, typically by
# looking at the top 20 results instead of the top 100 results.

# - With the first 2,000 popular appIDs, 12 apps can be recalled out of 22 apps.
# - With the first 7,000 popular appIDs, 16 apps.
# - With the first 13,000 popular appIDs, 19 apps.
# - With the first 18,500 popular appIDs, 21 apps.
# NB: the missing app is the DLC for Destiny 2, which cannot be recovered anyway
# as it is not a game. However, the base game appears in the 21 recalled apps.

#### Run

In [ ]:
from src.distance_utils import get_ground_truth_ranks, get_most_similar_app_ids
from src.score_utils import compute_distance_between_palettes

gift_index = 12
verbose = False

gift = egs_solutions["gift"][gift_index]
path_or_url = from_gift_to_egs_url(egs_solutions, gift)

reference_colors = prepare_image(path_or_url, params, verbose=verbose)

distances = compute_distance_between_palettes(
    reference_colors,
    palettes_subset,
    params,
)

most_similar_app_ids, indices = get_most_similar_app_ids(
    distances,
    app_ids_subset,
    params["topk"],
)

ground_truth_ranks = get_ground_truth_ranks(
    gift["appids"],
    most_similar_app_ids,
    )

#### Show the covers with the most similar color palettes

In [ ]:
from src.display_utils import display_results

max_num_displayed_images = 5

display_results(
    most_similar_app_ids,
    indices,
    distances,
    max_num_displayed_images,
    )

## Optimize the parameters

### Skip gift wrappings if the solutions are not popular enough

In [ ]:
test_egs_solutions = {
    "image": egs_solutions["image"],
    "gift": [
        e
        for e in egs_solutions["gift"]
        if any(str(app_id) in test_app_ids for app_id in e["appids"])
        ],
}

### Define the objective function

In [ ]:
from src.optuna_utils import my_objective

def objective(trial):
  return my_objective(
      trial,
      test_egs_solutions,
      palettes_subset,
      app_ids_subset,
      params,
      )

### Check the objective with user-input parameters

In [ ]:
from optuna.trial import FixedTrial

print(f"Possible color spaces: {COLOR_SPACES}")
print(f"Possible palette distances: {PALETTE_DISTANCES}")

params["color_space"] = "linear_hsv"
params["palette_distance"] = "custom_hausdorff_distance"

fixed_trial = FixedTrial({
    "exponent": 0.0,
    "factor": 0.0,
    })

objective(
      fixed_trial,
      )

### Run `optuna`

In [ ]:
from pathlib import Path

from src.optuna_utils import run_study

STUDY_FNAME = ""

study = run_study(
    objective,
    num_trials=100,
    study_fname = STUDY_FNAME,
    )